****Smart Travel Planner using Generative AI****

***Problem:***

Planning a trip takes a lot of time — from finding destinations to building a personalized itinerary. What if we could automate it with AI?

This project creates a **Smart Travel Planner** powered by **Generative AI**, which takes in user preferences (like destination, interests, and budget) and returns a day-by-day itinerary.

### ✅ GenAI Capabilities Demonstrated:

1. **Few-shot Prompting** – To generate personalized itineraries
2. **Structured Output (JSON/Table)** – Clean, formatted itinerary
3. **Image Understanding** – Suggest places from user-uploaded images
4. **Embeddings + Vector Search (RAG)** – Recommend similar destinations


🟩 **2. Install and Import Required Libraries**

In [1]:
import os
import json
import random
import torch
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display
from sentence_transformers import SentenceTransformer
from transformers import BlipProcessor, BlipForConditionalGeneration


2025-04-14 07:16:48.971118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744615009.235857      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744615009.320303      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🟩 **3. Load a Mini Travel Dataset**

In [2]:
# Sample synthetic travel dataset
data = [
    {"place": "Eiffel Tower", "location": "Paris", "category": "landmark", "desc": "Iconic wrought-iron lattice tower offering city views."},
    {"place": "Louvre Museum", "location": "Paris", "category": "museum", "desc": "Famous art museum with Mona Lisa."},
    {"place": "Shibuya Crossing", "location": "Tokyo", "category": "city life", "desc": "One of the world's busiest pedestrian crossings."},
    {"place": "Fushimi Inari Shrine", "location": "Kyoto", "category": "temple", "desc": "Famous for its thousands of vermilion torii gates."},
    {"place": "Colosseum", "location": "Rome", "category": "landmark", "desc": "Ancient Roman gladiator arena."},
    {"place": "Grand Bazaar", "location": "Istanbul", "category": "market", "desc": "One of the largest covered markets in the world."}
]

df = pd.DataFrame(data)
df.head()


,place,location,category,desc
0,Eiffel Tower,Paris,landmark,Iconic wrought-iron lattice tower offering cit...
1,Louvre Museum,Paris,museum,Famous art museum with Mona Lisa.
2,Shibuya Crossing,Tokyo,city life,One of the world's busiest pedestrian crossings.
3,Fushimi Inari Shrine,Kyoto,temple,Famous for its thousands of vermilion torii ga...
4,Colosseum,Rome,landmark,Ancient Roman gladiator arena.


In [3]:
#We simulate a few-shot prompt to generate a personalized itinerary based on user preferences. This approach mimics how LLMs learn from examples.
#python
# Sample user preferences
user_input = {
    "destination": "Tokyo",
    "days": 3,
    "interests": ["Anime", "Technology", "Food"],
    "budget": "Medium"
}

# Few-shot example prompt
example_prompt = f'''
Destination: Paris
Days: 2
Interests: History, Art, Food
Budget: High

Day 1:
- Morning: Visit Louvre Museum
- Afternoon: Lunch at Le Jules Verne, Eiffel Tower
- Evening: Seine River Cruise

Day 2:
- Morning: Notre-Dame Cathedral
- Afternoon: Montmartre Walking Tour
- Evening: Dinner at Le Meurice

---

Destination: {user_input["destination"]}
Days: {user_input["days"]}
Interests: {', '.join(user_input["interests"])}
Budget: {user_input["budget"]}
'''


In [4]:
# Simulated AI Response (Mocked)
itinerary = {
    "Day 1": {
        "Morning": "Visit Akihabara - Anime shopping",
        "Afternoon": "TeamLab Planets immersive digital art museum",
        "Evening": "Explore Shibuya Crossing and dine at Ichiran Ramen"
    },
    "Day 2": {
        "Morning": "Ghibli Museum in Mitaka",
        "Afternoon": "Odaiba and Miraikan Science Museum",
        "Evening": "Try conveyor belt sushi in Shinjuku"
    },
    "Day 3": {
        "Morning": "Tsukiji Outer Market Food Tour",
        "Afternoon": "Meiji Shrine and Harajuku",
        "Evening": "Sumida River Night Walk"
    }
}

In [5]:
# Display Itinerary as Table
pd.DataFrame(itinerary).T

,Morning,Afternoon,Evening
Day 1,Visit Akihabara - Anime shopping,TeamLab Planets immersive digital art museum,Explore Shibuya Crossing and dine at Ichiran R...
Day 2,Ghibli Museum in Mitaka,Odaiba and Miraikan Science Museum,Try conveyor belt sushi in Shinjuku
Day 3,Tsukiji Outer Market Food Tour,Meiji Shrine and Harajuku,Sumida River Night Walk


### 🔎 Explanation
This simulates how an LLM like ChatGPT or Gemini can take structured user preferences and generate personalized travel plans. Few-shot examples help guide the AI.

In [6]:
# 📦 Step 1: Install required libraries (uncomment if needed)
# !pip install -q transformers torchvision pillow requests

# 🧠 Step 2: Import dependencies
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import torch

# 📥 Step 3: Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# 🖼️ Step 4: Load the image from the given URL
img_url = "https://imgengine.khaleejtimes.com/khaleejtimes/import/images/eAOyHSOss2PmvEkt5gjg-RpsN5U.png?width=1200&format=auto"
image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# 📝 Step 5: Generate caption
inputs = processor(images=image, return_tensors="pt")
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

# 📢 Step 6: Display result
print("🖼️ Image Caption:", caption)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

🖼️ Image Caption: a col of a city with a lot of buildings


### 🔎 Explanation
BLIP combines vision + language models to describe what's in the image, helping users identify or remember places they want to visit.

In [7]:
# 📦 Step 1: Install required libraries (uncomment if needed)
# !pip install -q sentence-transformers

# 🧠 Step 2: Import necessary libraries
from sentence_transformers import SentenceTransformer, util
import torch

# 🌍 Step 3: Sample destinations to find similarity with
places = [
    "Eiffel Tower in Paris",
    "Mount Fuji in Japan",
    "Akihabara shopping in Tokyo",
    "Ginza luxury shopping street",
    "TeamLab Borderless Museum in Tokyo",
    "Shibuya Crossing at night"
]

# 🔄 Step 4: Create embeddings for the places
model = SentenceTransformer('all-MiniLM-L6-v2')  # Pretrained model for embeddings
embeddings = model.encode(places, convert_to_tensor=True)

# 🔍 Step 5: Define query for similar places
query = "digital immersive art experience"
query_emb = model.encode(query, convert_to_tensor=True)

# 📊 Step 6: Compute cosine similarity between query and places embeddings
cos_scores = util.cos_sim(query_emb, embeddings)[0]

# 🥇 Step 7: Find top 3 recommended places based on similarity
top_results = torch.topk(cos_scores, k=3)

# 📢 Step 8: Display top recommended places
print("\nTop recommended places:")
for score, idx in zip(top_results[0], top_results[1]):
    print(f"- {places[idx]} (Score: {score:.2f})")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Top recommended places:
- TeamLab Borderless Museum in Tokyo (Score: 0.19)
- Mount Fuji in Japan (Score: 0.12)
- Ginza luxury shopping street (Score: 0.10)


### 🔎 Explanation
Embeddings turn sentences into vectors. We can find similar experiences by comparing vector distances — a core part of Retrieval Augmented Generation (RAG).

In [8]:
## 📁 Step 5: Structured Output

#We return the full itinerary as a structured JSON, which can be saved or converted to a PDF.
#python
# Save to JSON
with open("tokyo_itinerary.json", "w") as f:
    json.dump(itinerary, f, indent=4)

## ✅ Final Output Summary
- User provided preferences → AI generates personalized itinerary
- Upload image → Get caption → Recommend similar spots
- Return all info in structured JSON format


## 🚀 Conclusion
Generative AI can simplify personal planning like travel:
- Helps automate repetitive tasks
- Personalizes user experience
- Works well with multimodal input (text + image)


## 🔮 Future Scope
- Add hotel & flight API integrations
- Support PDF/WhatsApp sharing of the itinerary
- Add speech-to-text for voice input (audio understanding)

## ⚠️ Limitations
- AI may hallucinate or generate invalid places
- Doesn’t validate availability or prices
- Should be used as an assistant, not a replacement for travel agents